### Import libraries and set up the splits

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from os import getcwd

print(tf.__version__)
print(tfds.__version__)

### Loab mobilenet model and its features

In [ ]:
import tensorflow_hub as hub 

model_selection = ("mobilenet_v2", 224, 1280) 
handle_base, pixels, FV_SIZE = model_selection
IMAGE_SIZE = (pixels, pixels)

filePath = f"{getcwd()}/data"

# You can also use directly to download from the source.

# MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)
# feature_extractor = hub.KerasLayer(MODULE_HANDLE,
#                                    input_shape=IMAGE_SIZE + (3,))

# The data is already downloaded for you at the filepath

feature_extractor = hub.KerasLayer(filePath+'/mobilenet_v2_feature_vector',input_shape=IMAGE_SIZE + (3,))
feature_extractor.trainable = True  

### Split the dataset

In [ ]:
# EXERCISE: Split the dataset

splits = ["train[:10%]", "train[-5%:]", "train[-10%:-5%]"] #DESCRIBE YOUR SPLITS HERE#]
    
# Remember to use `cats_vs_dogs:4.*.*` 
# https://www.tensorflow.org/datasets/catalog/cats_vs_dogs
    
# It has been downloaded for you so use the data_dir parameter 
# else it will try to download the dataset and give you an error here

splits, info = tfds.load(name='cats_vs_dogs:4.*.*', 
                         data_dir=filePath, 
                         split=splits,
                         with_info=True) #YOUR CODE HERE)

(train_examples, validation_examples, test_examples) = splits

# Testing lengths of the data if they are loaded correctly. Do not edit the code below
    
train_len = len(list(train_examples))
validation_len = len(list(validation_examples))
test_len = len(list(test_examples))
print(train_len)
print(validation_len)
print(test_len)

### Shuffle and map the new batches

In [ ]:
num_examples = 500 # DO NOT CHANGE THIS FOR THE GRADER, UPDATE AND USE IT WHEN PLAYING AROUND AND TRAINING IT LOCALLY.
num_classes = 2


def format_image(features):
    image = features['image']
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, features['label']

BATCH_SIZE =  16 # DO NOT EDIT

# For training batches, shuffle the examples by num_examples, map using the function defined above
# and batching using the batch_size.

# For validation and test batches, just avoid shuffling and follow the rest as training batch example

train_batches = train_examples.map(lambda features: format_image(features)).shuffle(num_examples).batch(BATCH_SIZE) #YOUR CODE HERE 
validation_batches = validation_examples.map(lambda features: format_image(features)).batch(BATCH_SIZE) #YOUR CODE HERE 
test_batches = test_examples.map(lambda features: format_image(features)).batch(BATCH_SIZE) #YOUR CODE HERE 

### Define your transfer learning model

In [ ]:
# EXERCISE: Define the model

# The new model will take the features from the mobilenet via transfer learning
# And add a new dense layer at the bottom, with 2 classes -- for cats and dogs

model = tf.keras.Sequential([
        feature_extractor,
        tf.keras.layers.Dense(2, activation="softmax")
        #YOUR CODE HERE, # USE the feature extractor which you loaded before
        #YOUR CODE HERE # Define a keras dense layer with 2 classes and softmax activation
])

# Model summary to test your model layers, output shape and number of parameters.
model.summary()

### Training your model

In [ ]:
# # Compile the model with adam optimizer and sparse categorical crossentropy, 
# # and track the accuracy metric

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

# # Train it for a number of epochs. You should not need many (max 5)
# # Train on the train_batches, and validation on the validation_batches 

EPOCHS = 5 # YOUR CODE HERE

history = model.fit(train_batches, epochs=EPOCHS, validation_data=validation_batches,verbose=1)

model.summary()

In [ ]:
# # Evaluate the model on the test batches
eval_results = model.evaluate(test_batches)

# # And print the results. 
for metric, value in zip(model.metrics_names, eval_results):
     print(metric + ': {:.4}'.format(value))